In [ ]:
import micropip
await micropip.install('folium')

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

In [ ]:
gdf = gpd.read_file("data/pd_neighborhoods_datasd.geojson")
gdf 

In [ ]:
gdf["name"].unique() 

In [ ]:
print(gdf["name"].value_counts().to_string())

In [ ]:
gdf.plot(figsize=(30, 30), edgecolor='black', column='name', legend=True)
plt.title("San Diego Police Neighborhoods")
plt.show()

In [ ]:
# Get center of map
center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]

# Create folium map
m = folium.Map(location=center, zoom_start=11)

# Add polygons
#folium.GeoJson(gdf).add_to(m)
folium.GeoJson(
    gdf,
    name="Neighborhoods",
    tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Neighborhood:"])
).add_to(m)

# Display
m